In [1]:
import pandas as pd
import numpy as np

In [2]:
# Célula de código do python
try:
    print('Obtendo dados...')

    # Constante dos dados
    ENDEREÇO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    # Obtendo dados e delimitando
    df = pd.read_csv(ENDEREÇO_DADOS, sep=',', encoding='utf-8')[['review_title','overall_rating']]

    # Excluindo dados que não existem (NaN) # equivalente a dados nulos null
    df = df.dropna(subset=['review_title','overall_rating'])

    # Transformando colunas em arrays
    texts = np.array(df['review_title'])
    rating = np.array(df['overall_rating'])

    print(df['overall_rating'].unique()) # exibir categorias
    print(df['overall_rating'].value_counts()) # exibir quantidade de valores por categoria

    print(df.head())
except Exception as e:
    print('Erro ao obter dados: ',e)

Obtendo dados...
[4 5 1 2 3]
overall_rating
5    47856
4    32285
1    27280
3    16279
2     8371
Name: count, dtype: int64
                       review_title  overall_rating
0                               Bom               4
1  Preço imbatível, ótima qualidade               4
2      ATENDE TODAS AS EXPECTATIVA.               4
3        presente mais que desejado               4
4            Sem duvidas, excelente               5


C:\Users\36131872024.1\AppData\Local\Temp\ipykernel_27248\1135367106.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENDEREÇO_DADOS, sep=',', encoding='utf-8')[['review_title','overall_rating']]


In [3]:
# Bibioteca para trabalhar com redes neurais artificiais
# Tersorflow: https://www.tensorflow.org/?hl=pt-br
from tensorflow.keras.preprocessing.text import Tokenizer # type:ignore  # Tokenizar
from tensorflow.keras.preprocessing.sequence import pad_sequences # type:ignore  # Ajustar tamanho do vetor
try:
    print("Vetorizando texto...")

    # Passo 1: tokenizar
    tokenizer = Tokenizer() 
    # transforma todas as palavras do texto em tokens e cria um dicionário com índices das palavras
    
    # Passo 2: Criar o dicionário
    # fit_on_texts: Cria o vocabulário, através do dicionário
    # associando cada token a um índice numérico a cada palavra
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo índice numérico.
    tokenizer.fit_on_texts(texts)

    # Passo 3: Vetorizar, ou seja, transformar os tokens em números,
    # a partir do dicionário criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)
    print(vetores)

    # Passo 4: Padronização do tamanho do vetor - pad (adicina zeros 0)
    padded_vetores = pad_sequences(vetores)

    print(padded_vetores) # Vetorzão (Aula teórica)
    print('Textos vetorizados com sucesso!')

except Exception as e:
    print('Erro ao vetorizar texto: ',e)
    

Vetorizando texto...
[[3], [63, 2620, 30, 16], [75, 349, 45, 155], [168, 48, 18, 2204], [53, 665, 7], [1, 2205], [9], [5, 97, 8, 1], [5, 8, 1], [4, 14, 6, 1], [406], [42], [23, 6, 1], [2364], [71], [6, 7237], [2365, 13, 6, 286], [2, 3], [2, 3], [250, 1265, 1699], [5, 8, 1], [52, 369], [37], [37], [2, 3, 570, 10, 214], [47, 4, 14, 6, 1], [9, 1], [6, 44], [6, 1, 56, 853, 1700, 1036, 382], [748, 13, 1151, 10, 25, 66], [1, 9, 11, 17, 81], [4, 14, 29, 56, 853], [5, 2], [36, 1, 11, 112, 35, 401, 73], [1, 2, 3], [1, 215], [5, 8, 7238], [7], [5, 2, 8, 1], [3, 1], [174, 35, 155], [9], [521], [5, 2, 8, 1, 11, 35, 501, 13, 18, 56, 51], [7], [27, 33], [325], [7, 1], [6, 1, 56, 51, 174, 8, 67], [2, 24, 6, 1], [2, 175], [66], [89, 544], [128], [4, 5, 8, 1], [298, 2, 3], [301, 579, 823], [4, 14, 6, 1], [3, 34, 121, 2206], [21], [195, 103], [156], [3], [3], [202, 153], [5, 2, 8, 1], [5, 97], [5], [480], [29, 4, 14], [44, 18, 6, 124], [2207, 122, 63], [169, 18, 178, 44], [9, 1], [9, 1], [261, 1793], [3

In [4]:
from tensorflow.keras.models import Sequential # type:ignore # Definir o modelo de rede neural utilizada
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout # type:ignore  # Camadas da rede neural
# Embedding: Classe do tensorflow
# Dropout ******* AJUSTE DE MODELO ********: função minimizar o overfitting

from tensorflow.keras.optimizers import Adam # type:ignore # otimizador de taxa de aprendizado
try:
    print('Contruindo a rede neural...')

    # Constantes do modelo:

    # 1ª Constante: Tamanho do vocabulário (tamanho do dicionário)
    VOCAB_SIZE = len(tokenizer.word_index)+1 # +1 pois começa em 0
    print(VOCAB_SIZE)
    # 2ª Constante: Tamanho máximo da sequência
    # É o comprimento máximo de um texto
    MAX_SEQUENCE_LENGHT = padded_vetores.shape[1] # shape: tamanho das colunas dentro do vetor
    # [   0    0    0 ...    0    0    3]

    # 3ª Constante: Tamanho do vetor de entrada
    # A literatura recomenda que inicia-se por uma quantidade igual a raiz quadrada do tamanho do vocabulário
    # Se o volume de dados for de larga escala, pode-se testar iniciando com um tamanho maior
    # Se o volume de dados for muito pequeno, pode-se testar iniciando com um tamanho menor
    # Cuidado com o overfitting, quando o modelo aprende de mais e começa a perder a 
    # capacidade de generalizar melhor, ou seja, observar todas as diferenças textuais.
    # Overfitting pode ser observado no treino da rede neural.
    VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE)) # raiz do tamanho do vocabuário

    # Construção da rede neural (de regressão). Modelo sequencial
    # Sequential é um fluxo linear de camadas (conforme visto na Aula 02_RNA.pptx)
    # São processadas em ordem
    model = Sequential()

    # Camada de entrada
    # Embeddings, na qual os vetores de texto são inseridos
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=VETOR_LENGHT, input_length=MAX_SEQUENCE_LENGHT))

    # Camada oculta ou intermediária
    # LSTM - Long short-term memory, "memória de curto e longo prazo"
    # É onde o modelo treina, baseado nos seus vetores
    # Números de unidades de memória, que basicamente á quantidade de "neurônios"
    # Quanto maior a quantidade de neurônios, maior a acurácia, maior o custo de processamento
    # Só é necessário ter uma camada para ter uma saída. Podem ter mais camadas.
    # Primeiro TESTE, somente com 1 camada. Cuidado com o overfitting.
    
    # Primeira camada oculta
    model.add(LSTM(128)) # Uma camada intermediária terá 128 neurônios
    # Para adicionar mais camadas, basta repetir o comando model.add(LSTM(128))
    # É recomendado adicionar apenas uma camada de início. 
    # Após rodar o código, se necessário, vai adicionando novas camadas.
    # LSTM é um tipo de camada oculta. Existem outras.
    
    # Próximas camadas ocultas...
    #model.add(LSTM(128))

    # Camada de Saída - Camada Densa
    # Modelo de Classificação: Precisa ajustar para a quantidade de classes (categorias)
    # 5 Camadas. É esse o modelo do código exemplo03.py
    # Categorias: 1, 2, 3, 4 e 5
    # Modelo de Classificação: tem mais camadas de saída
    
    # ********* AJUSTE DE MODELO ***********
    # Em Redes Neurais Artificiais de Classificação, a função de ativação
    # mais utilizada é a softmax
    # Função de Ativação é um cálculo matemático que vai determinar a saída de cada neurônio
    # A Softmax é uma função que transforma os valores de saída em PROBABILIDADES que vão de 0 a 1
    # A soma das classes, no nosso caso 5 classes, será igual a 1
    
    # A outra função mais utilizada é a ReLu (que também pode ser utilizada em classificação)
    # É uma função que retorna 0 zero para saídas negativas e o valor original para saída maiores que 0 zero.
    # O que faz ela ser mais utilizada em RNA de Regressão
   
    model.add(Dense(5, activation='softmax'))
    # model.add(Dense(1, activation='relu')) # Ativação para RNA de Regressão Linear


    # Dropout é uma técnica de regularização do resultado, para minimizar o overfitting (alta aprendizagem)
    # Ele desabilita neurônios aleatóriamente, justamente para minimizar o overfitting
    # Cuidado com underfitting (baixa aprendizade), se reduzir demais é ruim!
    model.add(Dropout(0.05)) # desativar 5% dos neurônios, aleatóriamente
    # Fazer dropout se necessário


    # Construir o modelo
    # É literalmente pegar as definições anteriores e construir o modelo
    # input_shape: é o formato de dados de entrada e ainda o tamanho máximo do texto(MAX_SEQUENCE_LENGHT)
    # None: o modelo verifica o dado de entrada e define o formato
    model.build(input_shape=(None,MAX_SEQUENCE_LENGHT))

    # Otimizador de taxa de aprendizado
    # importante para ajustar em casos de overfitting
    # Adam: É o otimizador que vai ajustar essa taxa de aprendizado
    # learning_rate: Quanto menor, melhor o aprendizado. Menos risco de overfitting.
    otimizador = Adam(learning_rate=0.0001) # Reduzindo para compensar o desbalanceamento das classes nos dados de entrada

    # Compilar o modelo
    # Verificar se possui algum erro
    # Além disso vamos informar o otimizador e a nossa métrica de perda (loss)
    # Usada para avaliar o modelo. Ver se existe overfitting ou não
    # loss - sparse_categorical_crossentropy
    model.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')

    model.summary()
    print('Modelo configurado e criado com sucesso!')

except Exception as e:
    print('Erro ao construir a rede neural: ',e)
    

Contruindo a rede neural...
15620


C:\Users\36131872024.1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 31, 124)        │     1,936,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       129,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,067,061 (7.89 MB)

 Trainable params: 2,067,061 (7.89 MB)

 Non-trainable params: 0 (0.00 B)

Modelo configurado e criado com sucesso!


In [5]:
from sklearn.model_selection import train_test_split
# Treinar o modelo
try:
    print('Treinar o modelo de rede neural')

    X_train, X_test, y_train, y_test = train_test_split(
        padded_vetores,
        rating,
        test_size=0.2,
        random_state=42
    )
    
    # Para RNA de Classificação:
    # Quando construimos uma rede neural de classificação, os rótulos que a RNA atribui, não começa em 1, começa em zero 0.
    # Sendo que os dados são categorizados em 1, 2, 3, 4 e 5. 
    # Necessário ajustar as saídas (os y's)
    y_train_adjusted = y_train -1
    y_test_adjusted = y_test -1

    # ********* AJUSTE DE MODELO ***********
        # Aplicar pesos nas categorias
    # Soma do tamanho do dataframe dividido por cada categoria
    # A categoria com menor quantidade de registros receberá o maior peso para balancear o aprendizado.
    pesos = len(df['overall_rating']) / df['overall_rating'].value_counts()

    print(pesos)

    # o treino da rede neural
    model.fit(
        X_train, 
        y_train_adjusted,
        epochs=5, # Épocas de estudo da rede neural. Inicia com 5 e vai avaliando. 
        # Quanto mais epochs, mais aprendizado (mais processamento), porém cuidado com o overfitting.
        # A regra é a experimentação, avaliando a perda de treino com a perda de teste.
        # A busca é: loss (perda de treino) menor que val_loss (perda de teste).
        batch_size=32, # Bloco de dados de treino. # Quando maior o tamanho da batch, tende menor o aprendizado, porém evita overfitting
        # Quanto menor o tamanho da batch, tende a um maior aprendizado
        
        # ********* AJUSTE DE MODELO ***********
        class_weight = pesos.to_dict(), # Precisa receber os pesos em uma estrutura de dados do tipo dicionário
        
        validation_data=(X_test, y_test_adjusted)
    )
    # No caso da classificação, o loss precisa diminuir para não indicar overfitting

except Exception as e:
    print('Erro ao treinar o modelo de rede neural: ',e)
    
    

Treinar o modelo de rede neural
overall_rating
5     2.759758
4     4.090785
1     4.841312
3     8.112968
2    15.777207
Name: count, dtype: float64
Epoch 1/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - loss: 11.4382 - val_loss: 1.1504
Epoch 2/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 53s 16ms/step - loss: 9.9013 - val_loss: 1.1574
Epoch 3/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 72s 22ms/step - loss: 9.7529 - val_loss: 1.1034
Epoch 4/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 78s 24ms/step - loss: 9.6335 - val_loss: 1.1329
Epoch 5/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 75s 23ms/step - loss: 9.3303 - val_loss: 1.0942


In [6]:
  # Textos novos inseridos para realizar a previsão, baseada no modelo de rede neural treinado.
# Nesse momento, você pode conectar em uma rede de dados (instagram...), para realizar as previsões de sentimento.

novos_textos = ['Muito bom, gostei bastante. Top demais! Compensa muito!',
                'Não recomendo, péssimo produto. Não funciona.']

# Vetorizar
novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
novas_sequencias_padded = pad_sequences(novas_sequencias)

# Análise preditiva
predicoes = model.predict(novas_sequencias_padded)

# Formatar valores de saída (retirar a saída em hexadecimal)
np.set_printoptions(suppress=True, precision=4)

print('Previsões: ', predicoes)

# Resultado em escala de 1 a 5. Sendo 1 ruim e 5 bom.
# A probabilidade apresentada não é percentual.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Previsões:  [[0.0003 0.0003 0.0037 0.1732 0.8226]
 [0.8383 0.0864 0.033  0.0162 0.0262]]
